In [52]:
import pandas as pd
import numpy as np
import os

In [53]:
csvfile_1 = "Hawaii_Measures_Clean.csv"
csvfile_2 = "Hawaii_Stations_Clean.csv"

In [54]:
df1 = pd.read_csv(csvfile_1, dtype = object)
df2 = pd.read_csv(csvfile_2, dtype = object)
df1 = df1.drop(['Unnamed: 0'], axis = 1)
df2 = df2.drop(['Unnamed: 0'], axis = 1)
df1.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.0,63
2,USC00519397,2010-01-03,0.0,74
3,USC00519397,2010-01-04,0.0,76
4,USC00519397,2010-01-07,0.06,70


In [55]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime
from datetime import datetime

In [56]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [57]:
conn = engine.connect()

In [58]:
Base = declarative_base()
    
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key = True)
    date = Column(Text)
    prcp = Column(Float)
    station = Column(Text)
    tobs = Column(Float)
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [59]:
Base.metadata.create_all(engine)

In [60]:
df_of_data_to_insert_1 = pd.read_csv(csvfile_1, dtype = object).drop(['Unnamed: 0'], axis = 1)
df_of_data_to_insert_2 = pd.read_csv(csvfile_2, dtype = object).drop(['Unnamed: 0'], axis = 1)
#df_of_data_to_insert_1['date'] = pd.to_datetime(df_of_data_to_insert_1['date'])

In [61]:
data_1 = df_of_data_to_insert_1.to_dict(orient='records')
data_2 = df_of_data_to_insert_2.to_dict(orient='records')
data_1[0]

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [62]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [63]:
table_1 = sqlalchemy.Table('measurement', metadata, autoload=True)
table_2 = sqlalchemy.Table('station', metadata, autoload=True) 

In [64]:
table_2

Table('station', MetaData(bind=Engine(sqlite:///hawaii.sqlite)), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station', TEXT(), table=<station>), Column('name', TEXT(), table=<station>), Column('latitude', FLOAT(), table=<station>), Column('longitude', FLOAT(), table=<station>), Column('elevation', FLOAT(), table=<station>), schema=None)

In [65]:
conn.execute(table_1.insert(), data_1)
conn.execute(table_2.insert(), data_2)

In [66]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, '2010-01-01', 0.08, 'USC00519397', 65.0),
 (2, '2010-01-02', 0.0, 'USC00519397', 63.0),
 (3, '2010-01-03', 0.0, 'USC00519397', 74.0),
 (4, '2010-01-04', 0.0, 'USC00519397', 76.0),
 (5, '2010-01-07', 0.06, 'USC00519397', 70.0)]